# 🚀 Day 3-2: 앙상블 심화 (Bagging, Voting, Boosting)

앙상블(Ensemble) 학습의 철학은 이 한 문장으로 요약될 수 있습니다. 

단 하나의 강력한 모델에 의존하기보다, 여러 개의 평범한 모델(약한 학습기, weak learner)을 모아 집단 지성을 형성했을 때 훨씬 더 안정적이고 뛰어난 예측 성능을 얻을 수 있습니다.

Day 1에서 우리는 앙상블의 대표주자인 `RandomForest`를 이미 만나보았습니다. 

랜덤 포레스트는 <u>배깅(Bagging)</u> 이라는 앙상블 기법을 기반으로 만들어졌죠.

이번 시간에는 여기서 한 걸음 더 나아가, 배깅의 또 다른 변형인 <u>Extra Trees</u>를 알아보고, 

서로 다른 종류의 모델들을 결합하는 <u>보팅(Voting)</u>, 그리고 이전 모델의 실수를 보완하며 순차적으로 학습하는 <u>부스팅(Boosting)</u> 계열의 알고리즘들을 깊이 있게 탐색합니다.

이번 실습에서는 다음 두 데이터셋을 주로 사용합니다.

* <u>Heart Failure Records:</u> 환자의 임상 데이터를 기반으로 심부전으로 인한 사망 여부를 예측하는 분류 문제 (Bagging, Voting 실습용)

* <u>Seoul Bike Sharing Demand:</u> 서울시 공공자전거 '따릉이'의 시간대별 대여 수요를 예측하는 회귀 문제 (Boosting 실습용)

이 과정을 통해 여러분은 개별 모델의 성능을 뛰어넘어, 캐글(Kaggle)과 같은 데이터 경진대회에서 상위권을 차지하는 모델들이 어떻게 구성되는지에 대한 실질적인 감각을 익히게 될 것입니다.

---


### 1. Bagging 계열 심화 - Extra Trees

배깅은 <u>B</u>ootstrap <u>Agg</u>regat<u>ing</u>의 약자로, 

원본 데이터셋에서 무작위로 복원을 허용하여 여러 개의 부분 데이터셋(Bootstrap sample)을 만들고,

각 데이터셋으로 개별 모델을 학습시킨 뒤 결과를 종합하는 방식입니다. 

`RandomForest`는 여기에 추가로 각 노드를 분할할 때 특성의 일부만 무작위로 선택하는 과정을 더해 모델의 다양성을 극대화합니다.

이제 `RandomForest`와 매우 유사하지만, '극단적인 무작위성'을 더한 <u>Extra Trees (Extremely Randomized Trees)</u> 를 알아보겠습니다.

#### 🧠 개념 이해하기

`ExtraTrees`는 `RandomForest`와 두 가지 핵심적인 차이점을 가집니다.

1.  <u>부트스트랩 미사용 (No Bootstrapping):</u> `RandomForest`는 복원 추출을 통해 부트스트랩 샘플을 만들지만, `ExtraTrees`는 기본적으로 전체 원본 훈련 데이터셋을 모든 트리에 그대로 사용합니다 (`bootstrap=False`가 기본값). 이는 데이터의 편향(Bias)을 약간 줄이는 효과를 가져올 수 있습니다.

2.  <u>극도의 무작위 분할 (Extremely Randomized Splits):</u> `RandomForest`는 각 노드에서 최적의 분할 지점을 찾기 위해 고려 대상이 된 특성들의 모든 가능한 분할 지점을 평가합니다. 반면, `ExtraTrees`는 한 단계 더 나아가 분할 지점(threshold)마저도 <u>무작위로</u> 선택합니다. 즉, 랜덤한 특성 서브셋에 대해 랜덤한 분할 후보들을 생성하고, 그중에서 가장 좋은 것을 선택합니다.

이러한 극단적인 무작위성은 다음과 같은 효과를 가져옵니다.

* <u>분산 감소:</u> 트리의 분할이 데이터에 덜 의존적이 되므로, 모델의 분산(Variance)이 줄어들어 과적합 방지에 더 효과적일 수 있습니다.

* <u>속도 향상:</u> 최적의 분할을 일일이 계산하지 않으므로 훈련 속도가 일반적으로 더 빠릅니다.

결론적으로 `ExtraTrees`는 약간의 편향 증가를 감수하는 대신, 분산을 크게 줄이고 훈련 속도를 높이는 전략을 취하는 모델이라고 할 수 있습니다.

#### 💻 코드로 알아보기

`Heart Failure Records` 데이터셋으로 `ExtraTreesClassifier`를 직접 사용해 보겠습니다.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# 데이터셋 로드
path = '../datasets/ml/heart-failure/heart_failure_clinical_records_dataset.csv'
df_heart = pd.read_csv(path)

In [2]:
# 특성(X)과 타겟(y) 분리
X = df_heart.drop('DEATH_EVENT', axis=1)
y = df_heart['DEATH_EVENT']

# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ExtraTreesClassifier 모델 생성 및 학습
# n_estimators: 트리의 개수
# random_state: 재현성을 위한 시드 고정
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
et_clf.fit(X_train, y_train)

# 예측 및 평가
y_pred = et_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"ExtraTreesClassifier 정확도: {accuracy:.4f}")
print(f"ExtraTreesClassifier F1 점수: {f1:.4f}")

# 특성 중요도 시각화
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': et_clf.feature_importances_})
feature_importances = feature_importances.sort_values('importance', ascending=False)

fig = px.bar(feature_importances,
             x='importance',
             y='feature',
             orientation='h',
             title='Extra Trees 특성 중요도',
             labels={'importance': '중요도', 'feature': '특성'})
fig.show()

ExtraTreesClassifier 정확도: 0.7333
ExtraTreesClassifier F1 점수: 0.4667


`time`(추적 기간), `ejection_fraction`(박출 계수), `serum_creatinine`(혈청 크레아티닌) 등이 사망 사건 예측에 중요한 변수임을 확인할 수 있습니다.

#### ✏️ 연습문제 1

`ExtraTreesClassifier`의 하이퍼파라미터를 조정하여 성능 변화를 관찰해보세요. `n_estimators`를 200으로, `max_depth`를 10으로 설정했을 때의 정확도와 F1 점수는 어떻게 변하는지 확인해보세요.

---


### 2. Hard & Soft Voting

Voting은 여러 다른 종류의 모델(Heterogeneous Models)을 묶어 예측을 수행하는 가장 간단하면서도 효과적인 앙상블 기법 중 하나입니다. 

예를 들어, 로지스틱 회귀, K-NN, 결정 트리 모델을 각각 만들어 이들의 "투표"로 최종 결정을 내리는 방식입니다.

#### 🧠 개념 이해하기

Voting에는 두 가지 주요 방식이 있습니다.

* <u>Hard Voting (다수결 투표):</u>
    * 가장 직관적인 방식으로, 각 모델이 예측한 클래스 레이블 중 가장 많이 나온 것을 최종 예측으로 선택합니다.
    
    * 예시: 3개의 분류기(A, B, C)가 있고, 어떤 샘플에 대해 A는 '1', B는 '0', C는 '1'로 예측했다면, 다수결에 따라 최종 예측은 '1'이 됩니다. 마치 민주주의 선거와 같습니다.

* <u>Soft Voting (가중치 투표):</u>
    * 각 모델이 예측한 <u>클래스별 확률</u>을 평균 내어, 가장 높은 확률을 가진 클래스를 최종 예측으로 선택합니다.
  
    * 이 방식을 사용하려면 모든 기본 모델이 클래스 확률을 예측하는 기능(`predict_proba()`)을 제공해야 합니다.
    * 예시: 3개의 분류기가 어떤 샘플에 대해 아래와 같이 확률을 예측했다면,
        * 모델 A: `P(Class=0)=0.1`, `P(Class=1)=0.9`
        
        * 모델 B: `P(Class=0)=0.6`, `P(Class=1)=0.4`
        * 모델 C: `P(Class=0)=0.4`, `P(Class=1)=0.6`
    * 평균 확률은 `P(Class=0) = (0.1+0.6+0.4)/3 = 0.367`, `P(Class=1) = (0.9+0.4+0.6)/3 = 0.633` 이므로, 최종 예측은 '1'이 됩니다.
    * 일반적으로 Soft Voting이 Hard Voting보다 성능이 더 좋다고 알려져 있습니다. 이는 모델이 예측에 얼마나 "확신"하는지에 대한 정보를 활용하기 때문입니다.

    <img src="https://blog.kakaocdn.net/dn/btzlAy/btrJC31chQi/m71MigUhKkBU9cErwvAL01/img.png" width="800">

#### 💻 코드로 알아보기

Day 2에서 사용했던 `Telco Customer Churn` 데이터셋을 다시 활용하여 여러 모델을 결합하는 `VotingClassifier`를 만들어 보겠습니다. 

이를 위해 먼저 간단한 전처리 파이프라인부터 구축합니다.

In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

# 데이터 로드 및 기본 전처리 (Day 2와 동일)
path = '../datasets/ml/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df_telco = pd.read_csv(path)
df_telco['TotalCharges'] = pd.to_numeric(df_telco['TotalCharges'], errors='coerce')
df_telco.drop('customerID', axis=1, inplace=True)
df_telco['Churn'].replace({'No': 0, 'Yes': 1}, inplace=True)

In [6]:
X = df_telco.drop('Churn', axis=1)
y = df_telco['Churn']

# 결측치가 있는 TotalCharges는 SimpleImputer로 처리
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X['TotalCharges'] = imputer.fit_transform(X[['TotalCharges']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 전처리 파이프라인 구축
numeric_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# 1. 개별 모델 정의
lr_clf = LogisticRegression(solver='liblinear', random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf = DecisionTreeClassifier(random_state=42)

# 2. VotingClassifier 생성
# Hard Voting
hard_voting_clf = VotingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='hard'
)

# Soft Voting
soft_voting_clf = VotingClassifier(
    estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='soft'
)

# 3. 전체 파이프라인으로 묶기
pipeline_hard = Pipeline([('preprocessor', preprocessor), ('classifier', hard_voting_clf)])
pipeline_soft = Pipeline([('preprocessor', preprocessor), ('classifier', soft_voting_clf)])

# 4. 학습 및 평가
pipeline_hard.fit(X_train, y_train)
pipeline_soft.fit(X_train, y_train)

hard_accuracy = pipeline_hard.score(X_test, y_test)
soft_accuracy = pipeline_soft.score(X_test, y_test)

print(f"Hard Voting 정확도: {hard_accuracy:.4f}")
print(f"Soft Voting 정확도: {soft_accuracy:.4f}")

Hard Voting 정확도: 0.7878
Soft Voting 정확도: 0.7615


Soft Voting이 Hard Voting에 비해 미세하게나마 성능이 더 좋은 것을 확인할 수 있습니다. 실제 문제에서는 이 차이가 더 크게 나타날 수 있습니다.

#### ✏️ 연습문제 2

`VotingClassifier`에 새로운 모델을 추가하여 성능 변화를 확인해보세요. `sklearn.svm.SVC` 모델(커널은 `rbf`, `probability=True`, `random_state=42`로 설정)을 Soft Voting 앙상블에 추가하고, 4개 모델을 사용했을 때의 정확도를 계산해보세요.

---

### 3. Boosting 심화

부스팅은 배깅과 달리, 순차적으로 모델을 학습시키는 방식입니다. 

각 모델은 이전 단계 모델의 <u>실수(오차)</u>를 보완하는 방향으로 학습됩니다. 

이 과정을 통해 점진적으로 성능이 개선되며, 매우 강력한 예측 모델을 만들 수 있습니다.

이번 파트에서는 부스팅의 고전적인 모델인 `AdaBoost`와 `GradientBoosting`, 그리고 캐글의 황제로 불리는 `XGBoost`의 핵심 튜닝 포인트를 알아보겠습니다.

#### 🧠 개념 이해하기

* <u>AdaBoost (Adaptive Boosting):</u>
    
    * <u>핵심 아이디어:</u> 이전 모델이 <u>잘못 예측한 샘플에 가중치</u>를 부여하여, 다음 모델이 해당 샘플에 더 집중하도록 만듭니다.
    
    * <u>동작 방식:</u>
        1.  모든 샘플에 동일한 가중치로 시작합니다.
        2.  첫 번째 약한 학습기(보통 결정 트리 스텀프, depth=1인 트리)를 학습하고 예측합니다.
        3.  예측이 틀린 샘플들의 가중치는 높이고, 맞힌 샘플들의 가중치는 낮춥니다.
        4.  가중치가 적용된 샘플들을 다시 학습하여 두 번째 모델을 만듭니다.
        5.  이 과정을 반복하고, 최종적으로 각 모델의 예측을 성능에 따라 가중합하여 최종 예측을 만듭니다.
    * <u>비유:</u> 오답 노트와 같습니다. 틀린 문제에 더 많은 시간을 할애하여 공부하는 것과 유사합니다.

* <u>GBRT (Gradient Boosting Regression Trees):</u>
    
    * <u>핵심 아이디어:</u> 이전 모델의 예측과 실제 값 사이의 <u>잔차(Residual Error)</u>를 다음 모델이 학습하도록 합니다.
    
    * <u>동작 방식 (회귀 예시):</u>
        1.  첫 번째 모델(보통 전체 타겟값의 평균)을 만듭니다.
        
        2.  실제 값과 첫 번째 모델의 예측값 사이의 오차(잔차)를 계산합니다.
        3.  이 <u>잔차 자체를 타겟으로</u> 하여 두 번째 모델을 학습시킵니다.
        4.  (첫 번째 모델 예측 + 두 번째 모델 예측)을 통해 새로운 예측을 만듭니다.
        5.  다시 새로운 예측과 실제 값 사이의 잔차를 계산하고, 이를 세 번째 모델이 학습합니다.
        6.  이 과정을 반복하며 점진적으로 실제 값에 근사해갑니다.
    * AdaBoost가 '틀린 샘플'에 집중한다면, Gradient Boosting은 '오차 자체'를 줄이는 데 집중하며, 손실 함수의 경사(Gradient)를 이용해 최적화하므로 더 일반적이고 강력합니다.

* <u>XGBoost 하이퍼파라미터 튜닝:</u>
    
    * `XGBoost`는 GBRT를 매우 효율적이고 확장 가능하게 구현한 라이브러리입니다. 성능을 최적화하기 위해 다음과 같은 파라미터 튜닝이 중요합니다.
    
    * `learning_rate` (또는 `eta`): 각 트리의 기여도를 조절하는 값입니다. 값을 낮추면 각 단계의 업데이트가 보수적으로 이루어져 과적합을 방지하지만, 그만큼 더 많은 `n_estimators`(트리 개수)가 필요합니다. (보통 0.01 ~ 0.3 사이)
    * `n_estimators`: 생성할 트리의 개수입니다. 너무 많으면 과적합, 너무 적으면 과소적합될 수 있습니다.
    * `max_depth`: 트리의 최대 깊이. 모델의 복잡도를 제어하는 핵심 파라미터입니다. (보통 3 ~ 10 사이)
    * <u>Early Stopping (조기 종료):</u> `n_estimators`를 넉넉하게 큰 값으로 설정하고, 검증 데이터(validation set)의 성능이 일정 반복 횟수(`early_stopping_rounds`) 동안 개선되지 않으면 학습을 자동으로 중단하는 매우 중요한 기능입니다. 이는 최적의 트리 개수를 찾아주고 과적합을 방지하는 효과적인 방법입니다.

#### 💻 코드로 알아보기

서울시 따릉이 수요 예측 데이터셋을 사용하여 부스팅 모델들을 비교하고 `XGBoost` 튜닝을 실습해 보겠습니다.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
import xgboost as xgb
import numpy as np

# 데이터 로드 (Kaggle Dataset)
# 출처: https://www.kaggle.com/datasets/saurabhshahane/seoul-bike-sharing-demand-prediction
# 이 데이터는 인코딩 문제로 cp949를 사용해야 할 수 있습니다.
df_bike = pd.read_csv('../datasets/ml/bike-sharing/SeoulBikeData.csv', encoding='cp949')


In [11]:
# 간단한 전처리
df_bike['Date'] = pd.to_datetime(df_bike['Date'], format='%d/%m/%Y')
df_bike['Month'] = df_bike['Date'].dt.month
df_bike['Day'] = df_bike['Date'].dt.day
df_bike = df_bike.drop('Date', axis=1)

# 범주형 변수 원-핫 인코딩
df_bike = pd.get_dummies(df_bike, columns=['Seasons', 'Holiday', 'Functioning Day'], drop_first=True)

# 특성(X)과 타겟(y) 분리
X = df_bike.drop('Rented Bike Count', axis=1)
y = df_bike['Rented Bike Count']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:

# --- AdaBoost ---
ada_reg = AdaBoostRegressor(n_estimators=50, random_state=42)
ada_reg.fit(X_train, y_train)
y_pred_ada = ada_reg.predict(X_test)
rmse_ada = np.sqrt(mean_squared_error(y_test, y_pred_ada))
print(f"AdaBoost RMSE: {rmse_ada:.4f}")


AdaBoost RMSE: 414.6277


In [13]:

# --- Gradient Boosting ---
gbrt_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
gbrt_reg.fit(X_train, y_train)
y_pred_gbrt = gbrt_reg.predict(X_test)
rmse_gbrt = np.sqrt(mean_squared_error(y_test, y_pred_gbrt))
print(f"Gradient Boosting RMSE: {rmse_gbrt:.4f}")


Gradient Boosting RMSE: 261.3390


In [14]:

# --- XGBoost with Early Stopping ---
# 검증을 위해 훈련 데이터를 다시 훈련용과 검증용으로 분리
X_train_xgb, X_val_xgb, y_train_xgb, y_val_xgb = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

xgb_reg = xgb.XGBRegressor(n_estimators=1000, # 넉넉하게 설정
                           learning_rate=0.05,
                           max_depth=5,
                           random_state=42,
                           early_stopping_rounds=50,# 50 라운드 동안 성능 개선이 없으면 중단
                           n_jobs=-1)

# 조기 종료를 설정하여 학습
xgb_reg.fit(X_train_xgb, y_train_xgb,
            eval_set=[(X_val_xgb, y_val_xgb)],
             
            verbose=False) # 학습 과정 출력 생략

y_pred_xgb = xgb_reg.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f"XGBoost (tuned with early stopping) RMSE: {rmse_xgb:.4f}")
print(f"XGBoost 최적 트리 개수: {xgb_reg.best_iteration}")

XGBoost (tuned with early stopping) RMSE: 219.2051
XGBoost 최적 트리 개수: 597


`XGBoost`가 다른 모델들보다 훨씬 낮은 RMSE(오차)를 보여주며, `early_stopping` 덕분에 1000개의 트리를 모두 사용하지 않고 262개에서 최적의 성능을 찾고 학습을 멈춘 것을 확인할 수 있습니다.

#### ✏️ 연습문제 3

위의 `XGBoost` 모델에서 `learning_rate`를 `0.1`로, `max_depth`를 `7`로 변경하여 다시 학습시켜보세요. RMSE와 최적 트리 개수(`best_iteration`)가 어떻게 변하는지 관찰하고, 왜 그렇게 변했는지 이유를 생각해보세요.